In [ ]:
!pip install pycaw

In [ ]:
import mediapipe as mp
import cv2
import time
import numpy as np
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [ ]:
wCam, hCam = 640,480

In [ ]:
class HandDetector():
    def __init__(self,mode=False,maxHands = 2,modelC = 1, detectionCon=0.5, trackCon= 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.modelC = modelC
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode,self.maxHands,self.modelC,self.detectionCon,self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def find_hands(self,img,draw= True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results =self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img
    def find_position(self,img,handNo=0,draw= True):
        landmarks_List=[]
        if self.results.multi_hand_landmarks:
            my_hand = self.results.multi_hand_landmarks[handNo]
            for handLms in self.results.multi_hand_landmarks:
                for id, lm in enumerate(my_hand.landmark):
                    # print(id, lm)
                    h, w, c = img.shape
                    cx, cy = int(lm.x*w), int(lm.y*h)
                    #print(id, cx, cy)#coordinates for eah landmark
                    landmarks_List.append([id, cx, cy])
                    if draw:
                        cv2.circle(img, (cx, cy),5, (255,0,255), cv2.FILLED)
        return landmarks_List              

In [ ]:
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.7)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_,
                            CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()#(-65.25, 0.0, 0.75) ---> the range from -65.25 to 0.0
#print('Volumme:',volume.GetVolumeRange())
volume.SetMasterVolumeLevel(-10.0, None)
minVol = volRange[0]
maxVol = volRange[1]
vol=0
#volBar = 0
while True:
    success, img=cap.read()
    img = detector.find_hands(img)
    landmarks_List = detector.find_position(img, draw=False)
    if len(landmarks_List) !=0:
        print(landmarks_List[4], landmarks_List[8])
        
        x1,y1 = landmarks_List[4][1], landmarks_List[4][2]#[x][y]
        x2,y2 = landmarks_List[8][1], landmarks_List[8][2]#[x][y]
        cx,cy = (x1+x2)//2 , (y1 + y2)//2
        
        cv2.circle(img, (x1,y1),10,(255,0,255),cv2.FILLED)
        cv2.circle(img, (x2,y2),10,(255,0,255),cv2.FILLED)
        cv2.line(img, (x1,y1), (x2,y2), (255,0,255),3)
        cv2.circle(img, (cx,cy),10,(255,0,255),cv2.FILLED)
        
        length = math.hypot(x2-x1 , y2-y1)
        #print(length)
        
        
        #Hand range 50 - 300
        #Volume Range -65 - 0
        vol = np.interp(length,[50,300],[minVol,maxVol])
        #volBar  = np.interp(length,[50,300],[400,150])
        print(int(length),vol)
        volume.SetMasterVolumeLevel(vol, None)
        
        if length<50:
            cv2.circle(img, (cx,cy),10,(0,255,0),cv2.FILLED)
            
    #cv2.rectangle(img, (50,150),(85,400),(0,255,0),3)
    #cv2.rectangle(img, int(vol),(85,400),(0,255,0),cv2.FILLED) 
    
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, str(int(fps)),(10,70),cv2.FONT_HERSHEY_PLAIN,3,
               (255,0,255),3)
    cv2.imshow("image",img)
    cv2.waitKey(1)